## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Moerai,-22.4333,-151.3333,74.12,81,13,13.51,PF,few clouds
1,1,Pemangkat,1.1667,108.9667,79.09,83,100,5.46,ID,overcast clouds
2,2,Saskylakh,71.9167,114.0833,31.28,98,100,6.73,RU,light snow
3,3,Basoko,1.2391,23.6160,71.92,88,58,0.63,CD,broken clouds
4,4,Ucluelet,48.9329,-125.5528,65.08,60,0,4.63,CA,clear sky


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp = float(input("What is the maximum temperature? "))
min_temp = float(input("What is the minimum temperature? "))

What is the maximum temperature? 90
What is the minimum temperature? 70


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_temp_df = city_data_df.loc[
            (city_data_df["Max Temp"] >= min_temp) & \
            (city_data_df["Max Temp"] <= max_temp)]

In [19]:
# 4a. Determine if there are any empty rows.
city_temp_df.count()

City_ID                315
City                   315
Lat                    315
Lng                    315
Max Temp               315
Humidity               315
Cloudiness             315
Wind Speed             315
Country                315
Current Description    315
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_temp_df.dropna()
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Moerai,-22.4333,-151.3333,74.12,81,13,13.51,PF,few clouds
1,1,Pemangkat,1.1667,108.9667,79.09,83,100,5.46,ID,overcast clouds
3,3,Basoko,1.2391,23.6160,71.92,88,58,0.63,CD,broken clouds
8,8,Jamestown,42.0970,-79.2353,80.02,46,40,10.00,US,scattered clouds
11,11,Hithadhoo,-0.6000,73.0833,81.86,75,100,8.48,MV,overcast clouds


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Moerai,PF,74.12,few clouds,-22.4333,-151.3333,
1,Pemangkat,ID,79.09,overcast clouds,1.1667,108.9667,
3,Basoko,CD,71.92,broken clouds,1.2391,23.6160,
8,Jamestown,US,80.02,scattered clouds,42.0970,-79.2353,
11,Hithadhoo,MV,81.86,overcast clouds,-0.6000,73.0833,
12,Finschhafen,PG,78.62,overcast clouds,-6.6024,147.8681,
13,Rikitea,PF,72.23,clear sky,-23.1203,-134.9692,
14,Lahaina,US,84.20,overcast clouds,20.8783,-156.6825,
16,Bambous Virieux,MU,71.85,light rain,-20.3428,57.7575,
17,Victoria,HK,84.38,few clouds,22.2855,114.1577,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [23]:
# 6b. Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [24]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
hotel_df = hotel_df.dropna()
hotel_df['Hotel Name'].value_counts()

Le Manotel                                      1
Amangalla                                       1
Auberge La Place de Jules                       1
Macs                                            1
Yos ega cv                                      1
                                               ..
Hampton Inn State College                       1
RedDoorz Syariah near Taman Berlabuh Tarakan    1
Inubosaki Kanko Hotel                           1
Pirineos                                        1
HOTEL BOUTIQUE SANTO DOMINGO                    1
Name: Hotel Name, Length: 115, dtype: int64

In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
df=hotel_df
hotel_df.to_csv(output_data_file, index_label="City_ID")
df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Moerai,PF,74.12,few clouds,-22.4333,-151.3333,Le Manotel
1,Pemangkat,ID,79.09,overcast clouds,1.1667,108.9667,Hotel Duyung Permai 2
11,Hithadhoo,MV,81.86,overcast clouds,-0.6000,73.0833,Scoop Guest House
12,Finschhafen,PG,78.62,overcast clouds,-6.6024,147.8681,Oasis Gagidu Ltd
13,Rikitea,PF,72.23,clear sky,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
656,Lengshuitan,CN,81.23,scattered clouds,26.4111,111.5956,Jialong Hotel
662,Betma,IN,72.63,few clouds,22.6833,75.6167,Khandelwal Paradise
665,Rawlins,US,73.29,broken clouds,41.7911,-107.2387,"Holiday Inn Express Rawlins, an IHG Hotel"
670,Lorengau,PG,80.85,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
hotel_df = pd.read_csv(output_data_file)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather/dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))